# Regularization - LASSO and Ridge Regression

Regularization is a method of penalizing extreme parameter weights in order to reduce overfitting, filter out noise from the data, and mitigate collinearity. The technique is used in regression modeling, whereby it introduces a penalty function that is is optimized in addition to the standard residual sum of squares. 

It is important to note that independent variables need to be scaled before performing regularization, or else the process will not function properly. 

The two primary methods of regularization are L1 and L2, where L1 regularization is used in LASSO regression and L2 regularization is used in Ridge Regression. 



## L2 Regularization with Ridge Regression

L2 Regularization adds a factor of sum of squares of coefficients in the optimization objective. Thus, ridge regression optimizes the following:

Objective (Cost) Function = RSS + λ * (sum of square of coefficients)

<img src="extras/Ridge1.png" width="700" height="700" />

Here, λ (lambda) is the parameter which balances the amount of emphasis given to minimizing RSS vs minimizing sum of square of coefficients. λ can take various values:

λ = 0:  
- The objective becomes same as simple linear regression.
- We’ll get the same coefficients as simple linear regression.  

λ = ∞:  
- The coefficients will be zero. Why? Because of infinite weightage on square of coefficients, anything less than zero will make the objective infinite.   

0 < λ < ∞:
- The magnitude of α will decide the weightage given to different parts of objective.
- The coefficients will be somewhere between 0 and ones for simple linear regression.

To better understand this how lambda affects the coefficients, et's follow the process of the regression algorithm. It tries to minimze the cost function above by performing a gradient descent. In order to determine the gradient for given weights, it must perform a partial derivative with respect to a particular weight (wj):

<img src="extras/Ridge2.png" width="700" height="700" />

We then update the jth weight by subtracting the learning rate times the gradient, resulting in:

<img src="extras/Ridge3.png" width="700" height="700" />

The first term on the right-hand side of the equation tells us that ridge regression is equivalent to reducing the weight by a factor of (1-2λη) first and then applying the same update rule as simple linear regression. Here we can see how different sizes of lambda can affect the weights of the coeffiecients, and how the coefficients can never fully be zero. 

## Ridge Regression Coding Example

We are going to use the Boston housing dataset to perform our examples

In [1]:
from sklearn import datasets
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
boston = datasets.load_boston()

In [3]:
X = pd.DataFrame(boston['data'], columns=boston['feature_names'])
y = pd.Series(boston['target'], name='MED')
bos = pd.concat([y, X], axis=1)

In [129]:
from sklearn.linear_model import Ridge

alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

coeffs = {}
z = 1.01
for alpha in alpha_ridge:
    r = Ridge(alpha=alpha, normalize=True)
    r = r.fit(X, y)
    coeffs[str(z)+':'+str(alpha)] = np.append(r.score(X,y), np.append(r.intercept_, r.coef_))
    z += .01

When we look at the coefficients for the different values of alpha, what we see is that their magnitude for every predictor (except for DIS at alpha=5 and RAD at alpha=1). As the impact of each predictor decreases, the model increases in simplicity, meaning underfitting begins to take place. We confirm this by looking at the model score, which begins to drop drastically after alpha reaches 1. 

In [128]:
a = pd.DataFrame(coeffs).transpose()
columns = ['SCORE', 'INTERCEPT']+list(X.columns)
dict = {}
i = 0 
for c in columns:
    dict[i] = c
    i += 1
a.rename(columns = dict, inplace=True)
a

,SCORE,INTERCEPT,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
1.01:1e-15,0.740608,36.491103,-0.107171,0.046395,0.020860,2.688561,-17.795759,3.804752,0.000751,-1.475759,0.305655,-0.012329,-0.953464,0.009393,-0.525467
1.02:1e-10,0.740608,36.491103,-0.107171,0.046395,0.020860,2.688561,-17.795759,3.804752,0.000751,-1.475759,0.305655,-0.012329,-0.953464,0.009393,-0.525467
1.03:1e-08,0.740608,36.491101,-0.107171,0.046395,0.020860,2.688561,-17.795757,3.804753,0.000751,-1.475759,0.305655,-0.012329,-0.953464,0.009393,-0.525467
1.04:0.0001,0.740608,36.471735,-0.107121,0.046362,0.020681,2.689258,-17.783439,3.805387,0.000739,-1.475111,0.305233,-0.012309,-0.953268,0.009392,-0.525373
1.05:0.001,0.740605,36.299120,-0.106679,0.046064,0.019094,2.695421,-17.673424,3.811020,0.000635,-1.469303,0.301487,-0.012127,-0.951519,0.009390,-0.524538
1.06:0.01,0.740344,34.724275,-0.102710,0.043381,0.005483,2.748085,-16.652015,3.860450,-0.000289,-1.413705,0.268783,-0.010573,-0.935255,0.009365,-0.516574
1.07:1,0.635014,21.026665,-0.059368,0.017702,-0.072405,2.311548,-3.927643,2.874591,-0.009297,-0.249665,-0.004520,-0.002736,-0.535678,0.006218,-0.261501
1.08:5,0.415215,23.554065,-0.037414,0.012091,-0.054343,0.964243,-2.661259,1.193735,-0.009043,0.018627,-0.026814,-0.002104,-0.249294,0.003280,-0.111930
1.09:10,0.297932,23.655470,-0.025734,0.008545,-0.038718,0.550144,-1.943350,0.708549,-0.006844,0.037962,-0.021554,-0.001513,-0.155247,0.002176,-0.068882
1.1:20,0.191601,23.380079,-0.015848,0.005351,-0.024335,0.295135,-1.243978,0.395494,-0.004452,0.032190,-0.014334,-0.000955,-0.089635,0.001315,-0.039586


Given that the prior exercise demonstrates an increasing model score with a decreasing alpha, while on the other increasing likelihood of overfitting, how do we know what the optimal level of alpha is? We can do so using grid search with cross validation. This will enable us to test different levels of alpha using k-fold cross-validation, which should give us unbiased score estimates.  

Once we perform that, we can see that grid search actually recommends and alpha of 1 as the optimal weight for our L2 regularization.

In [133]:
from sklearn.grid_search import GridSearchCV

grid_search = GridSearchCV(Ridge(alpha=alpha, normalize=True), param_grid={'alpha': alpha_ridge}, cv=10, n_jobs=-1)
grid_search.fit(X, y)

print grid_search.best_estimator_
print grid_search.score(X,y)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
   random_state=None, solver='auto', tol=0.001)
0.635014072122


# L1 Regularization with LASSO Regression

LASSO (Least Absolute Shrinkage and Selection Operator) regression relies on L1 regularization, which adds a factor of sum of absolute value of coefficients in the optimization objective. Thus, lasso regression optimizes the following:

Objective = RSS + λ * (sum of absolute value of coefficients)

<img src="extras/Lasso1.png" width="700" height="700" />

Here, λ (lambda, or alpha in sklearn) works similar to that of ridge and provides a trade-off between balancing RSS and magnitude of coefficients. Like that of ridge, λ can take various values. Lets iterate it here briefly:  

λ = 0:  

- Same coefficients as simple linear regression  

λ = ∞:  

- All coefficients zero (same logic as before)  

0 < λ < ∞: 

- Coefficients between 0 and that of simple linear regression

Because L1 relies one the absolute value of the coefficients, we aren't able to use gradient descent to optimize (meaning no learning rate). In this case, we have to use a different technique called as coordinate descent which is based on the concept of sub-gradients. In L1 regularization, our decision rule for a given weight is affected:

<img src="extras/Lasso2.png" width="700" height="700" />

Here g(w-j) represents (but not exactly) the difference between actual outcome and the predicted outcome considering all EXCEPT the jth variable. If this value is small, it means that the algorithm is able to predict the outcome fairly well even without the jth variable and thus it can be removed from the equation by setting a zero coefficient. This gives us some intuition into why the coefficients become zero in case of lasso regression, especially once lambda starts increasing. 

In coordinate descent, checking convergence is another issue. Since gradients are not defined, we need an alternate method. Many alternatives exist but the simplest one is to check the step size of the algorithm. We can check the maximum difference in weights in any particular cycle over all feature weights.

If this is lower than ‘tol’ specified, algorithm will stop. The convergence is not as fast as gradient descent and we might have to set the ‘max_iter’ parameter if a warning appears saying that the algorithm stopped before convergence. This is why we need to specify this parameter in the Lasso generic function.

## LASSO (Least Absolute Shrinkage and Selection Operator) Regression Coding Example

In [135]:
from sklearn.linear_model import Lasso

alpha_ridge = [1e-15, 1e-10, 1e-8, 1e-4, 1e-3,1e-2, 1, 5, 10, 20]

coeffs = {}
z = 1.01
for alpha in alpha_ridge:
    r = Lasso(alpha=alpha, normalize=True, max_iter=1e5)
    r = r.fit(X, y)
    coeffs[str(z)+':'+str(alpha)] = np.append(r.score(X,y), np.append(r.intercept_, r.coef_))
    z += .01

Again, when we look at the coefficients for the different values of alpha, what we see is that their magnitude for every predictor decreases as alpha increases. Starting at alpha=.001, we begin to see some cofficients drop to zero.  Interestingly, we don't see the model score drop too much as alpha increases. Additionally, starting with alpha=1, we see that every coefficient drops to zero. 

In [136]:
a = pd.DataFrame(coeffs).transpose()
columns = ['SCORE', 'INTERCEPT']+list(X.columns)
dict = {}
i = 0 
for c in columns:
    dict[i] = c
    i += 1
a.rename(columns = dict, inplace=True)
a

,SCORE,INTERCEPT,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
1.01:1e-15,0.740608,36.491103,-0.107171,0.046395,0.020860,2.688561,-17.795759,3.804752,0.000751,-1.475759,0.305655,-0.012329,-0.953464,0.009393,-0.525467
1.02:1e-10,0.740608,36.491103,-0.107171,0.046395,0.020860,2.688561,-17.795758,3.804752,0.000751,-1.475759,0.305655,-0.012329,-0.953464,0.009393,-0.525467
1.03:1e-08,0.740608,36.491086,-0.107170,0.046395,0.020860,2.688562,-17.795742,3.804753,0.000751,-1.475758,0.305655,-0.012329,-0.953463,0.009393,-0.525467
1.04:0.0001,0.740603,36.313495,-0.106436,0.045930,0.017759,2.691376,-17.632310,3.810254,0.000399,-1.471748,0.300900,-0.012090,-0.951010,0.009371,-0.524805
1.05:0.001,0.740277,34.859822,-0.099716,0.042336,0.000000,2.692627,-16.543743,3.847766,-0.000000,-1.416549,0.262504,-0.010242,-0.934032,0.009156,-0.523085
1.06:0.01,0.719520,22.428998,-0.035814,0.013100,-0.000000,2.354637,-8.567505,4.233876,-0.000000,-0.743384,0.000000,-0.000000,-0.819089,0.007275,-0.521000
1.07:1,0.000000,22.532806,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
1.08:5,0.000000,22.532806,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
1.09:10,0.000000,22.532806,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000
1.1:20,0.000000,22.532806,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,-0.000000,-0.000000,0.000000,-0.000000


Using grid search to find the optimal alpha, we get alpha=.01 as the best performing model.

In [137]:
from sklearn.grid_search import GridSearchCV

grid_search = GridSearchCV(Lasso(alpha=alpha, normalize=True), param_grid={'alpha': alpha_ridge}, cv=10, n_jobs=-1)
grid_search.fit(X, y)

print grid_search.best_estimator_
print grid_search.score(X, y)

Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
0.719519645326


## References

- https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/
- http://scikit-learn.org/stable/modules/linear_model.html#lasso
- http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
- http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso
- http://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/